In [ ]:
!nvidia-smi

Thu Jun 24 04:02:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/drive/MyDrive/EMNLP_Argmining2021

/content/drive/MyDrive/EMNLP_Argmining2021


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
import os
import re

### Loading the Dataset

In [ ]:
gold_data_dir = '/content/drive/MyDrive/EMNLP_Argmining2021/dataset/'

In [ ]:
def load_kpm_data(gold_data_dir, subset):
    arguments_file = os.path.join(gold_data_dir, f"arguments_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_{subset}.csv")
    #labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    #labels_file_df = pd.read_csv(labels_file)

    return arguments_df, key_points_df

In [ ]:
df_arguments,df_key_points,df_labels = load_kpm_data(gold_data_dir,'train')

In [ ]:
df_arguments_val,df_key_points_val = load_kpm_data(gold_data_dir,'dev')

In [ ]:
print(df_arguments.shape)
df_arguments.head()

(5583, 4)


,arg_id,argument,topic,stance
0,arg_0_0,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,-1
1,arg_0_1,A patient should be able to decide when they h...,Assisted suicide should be a criminal offence,-1
2,arg_0_2,a person has the right to end their suffering ...,Assisted suicide should be a criminal offence,-1
3,arg_0_3,a person should have the dignity to choose how...,Assisted suicide should be a criminal offence,-1
4,arg_0_4,a person should have the right to be able to c...,Assisted suicide should be a criminal offence,-1


In [ ]:
print(df_key_points.shape)
df_key_points.head()

(207, 4)


,key_point_id,key_point,topic,stance
0,kp_0_0,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1
1,kp_0_1,Assisted suicide reduces suffering,Assisted suicide should be a criminal offence,-1
2,kp_0_2,People should have the freedom to choose to en...,Assisted suicide should be a criminal offence,-1
3,kp_0_3,The terminally ill would benefit from assisted...,Assisted suicide should be a criminal offence,-1
4,kp_0_4,Assisted suicide allows people to solicit some...,Assisted suicide should be a criminal offence,1


In [ ]:
df_labels.head()

,arg_id,key_point_id,label
0,arg_0_0,kp_0_0,0
1,arg_0_121,kp_0_4,0
2,arg_0_121,kp_0_5,0
3,arg_0_121,kp_0_6,1
4,arg_0_121,kp_0_7,0


In [ ]:
argument_train = []
key_point_train = []
topic_train = []
stance_train = []

argument_val = []
key_point_val = []
topic_val = []
stance_val = []

In [ ]:
a = df_arguments[df_arguments['arg_id'] == 'arg_1_121'].argument.index[0]

In [ ]:
print(a)

367


In [ ]:
df_arguments[df_arguments['arg_id'] == 'arg_1_121'].stance[367]

-1

In [ ]:
df_arguments[df_arguments['arg_id'] == 'arg_0_0']['topic']

0    Assisted suicide should be a criminal offence
Name: topic, dtype: object

In [ ]:
df_key_points[df_key_points['key_point_id'] == 'kp_0_0']['key_point'][0]

'Assisted suicide gives dignity to the person that wants to commit it'

In [ ]:
text = 'arg_0_121'

int(re.split('_',text)[-1])

121

## Train and Validation files

In [ ]:
def give_tuples(df_labels,df_arguments,df_key_points):

  arguments = []
  key_points = []
  topics = []
  stance = []

  for i,j in zip(df_labels['arg_id'],df_labels['key_point_id']):

    
    idx_arg = df_arguments[df_arguments['arg_id'] == i ].argument.index[0]
    idx_kp = df_key_points[df_key_points['key_point_id'] == j ].key_point.index[0]

    arguments.append(df_arguments[df_arguments['arg_id'] == i]['argument'][idx_arg])
    topics.append(df_arguments[df_arguments['arg_id'] == i]['topic'][idx_arg])
    key_points.append(df_key_points[df_key_points['key_point_id'] == j]['key_point'][idx_kp])
    stance.append(df_key_points[df_key_points['key_point_id'] == j]['stance'][idx_kp])

  return arguments,key_points,topics, stance

In [ ]:
argument_train, key_point_train, topic_train,stance_train   = give_tuples(df_labels,df_arguments,df_key_points)

In [ ]:
df_labels['arg'] = argument_train
df_labels['key_point'] = key_point_train
df_labels['topic'] = topic_train
df_labels['stance'] = stance_train

df_labels.head()

,arg_id,key_point_id,label,arg,key_point,topic,stance
0,arg_0_0,kp_0_0,0,`people reach their limit when it comes to the...,Assisted suicide gives dignity to the person t...,Assisted suicide should be a criminal offence,-1
1,arg_0_121,kp_0_4,0,a cure or treatment may be discovered shortly ...,Assisted suicide allows people to solicit some...,Assisted suicide should be a criminal offence,1
2,arg_0_121,kp_0_5,0,a cure or treatment may be discovered shortly ...,Assisted suicide is akin to killing someone,Assisted suicide should be a criminal offence,1
3,arg_0_121,kp_0_6,1,a cure or treatment may be discovered shortly ...,Assisted suicide should not be allowed because...,Assisted suicide should be a criminal offence,1
4,arg_0_121,kp_0_7,0,a cure or treatment may be discovered shortly ...,Assisted suicide violates the doctor's role,Assisted suicide should be a criminal offence,1


In [ ]:
argument_val, key_point_val, topic_val,stance_val  = give_tuples(df_labels_val,df_arguments_val,df_key_points_val)

In [ ]:
df_labels_val['arg'] = argument_val
df_labels_val['key_point'] = key_point_val
df_labels_val['topic'] = topic_val
df_labels_val['stance'] = stance_val

df_labels_val.head()

,arg_id,key_point_id,label,arg,key_point,topic,stance
0,arg_4_121,kp_4_5,1,A real education is about giving students the ...,School uniform is harming the student's self e...,We should abandon the use of school uniform,1
1,arg_4_121,kp_4_6,0,A real education is about giving students the ...,School uniforms are expensive,We should abandon the use of school uniform,1
2,arg_4_121,kp_4_7,0,A real education is about giving students the ...,School uniforms are often uncomfortable/sexist,We should abandon the use of school uniform,1
3,arg_4_122,kp_4_8,0,children express themselves through the clothe...,School uniform harms learning/creativity,We should abandon the use of school uniform,1
4,arg_4_122,kp_4_5,1,children express themselves through the clothe...,School uniform is harming the student's self e...,We should abandon the use of school uniform,1


In [ ]:
print(df_labels.shape)
print(df_labels_val.shape)

(20635, 7)
(3458, 7)


In [ ]:
df_labels.to_csv('train.csv')
df_labels_val.to_csv('validation.csv')

### Prediction File Creation (development set predictions file)

In [ ]:
df_arguments_val.head()

,arg_id,argument,topic,stance
0,arg_4_0,having a school uniform can reduce bullying as...,We should abandon the use of school uniform,-1
1,arg_4_1,it is a good way to instill discipline,We should abandon the use of school uniform,-1
2,arg_4_2,it is cheaper for parents to buy school unifor...,We should abandon the use of school uniform,-1
3,arg_4_3,one of the benefits of school uniforms is that...,We should abandon the use of school uniform,-1
4,arg_4_4,researchers found that in schools across the w...,We should abandon the use of school uniform,-1


In [ ]:
df_key_points_val.head()

,key_point_id,key_point,topic,stance
0,kp_4_0,Children can still express themselves using ot...,We should abandon the use of school uniform,-1
1,kp_4_1,School uniform reduces bullying,We should abandon the use of school uniform,-1
2,kp_4_2,School uniforms encourage discipline or focus ...,We should abandon the use of school uniform,-1
3,kp_4_3,School uniforms saves costs,We should abandon the use of school uniform,-1
4,kp_4_4,School uniforms create a sense of equality/unity,We should abandon the use of school uniform,-1


In [ ]:
arg_pred = []
key_point_pred = []

arg_id_pred = []
key_point_id_pred = []
stance = []
topic = []

for arg,arg_id,topic_arg,stance_arg  in zip(df_arguments_val['argument'],df_arguments_val['arg_id'],df_arguments_val['topic'],df_arguments_val['stance']):
  for kp,kp_id,topic_kp,stance_kp in zip(df_key_points_val['key_point'],df_key_points_val['key_point_id'],df_key_points_val['topic'],df_key_points_val['stance']):
    if ( topic_arg == topic_kp and stance_arg == stance_kp):
      
      arg_pred.append(arg)
      arg_id_pred.append(arg_id)

      key_point_pred.append(kp)
      key_point_id_pred.append(kp_id)
      topic.append(topic_arg)

      stance.append(stance_arg)


In [ ]:
pred = pd.DataFrame({'arg_id':arg_id_pred,'key_point_id':key_point_id_pred,'argument':arg_pred, 'keypoint':key_point_pred, 'topic' : topic , 'stance': stance})

pred.head()

,arg_id,key_point_id,argument,keypoint,topic,stance
0,arg_4_0,kp_4_0,having a school uniform can reduce bullying as...,Children can still express themselves using ot...,We should abandon the use of school uniform,-1
1,arg_4_0,kp_4_1,having a school uniform can reduce bullying as...,School uniform reduces bullying,We should abandon the use of school uniform,-1
2,arg_4_0,kp_4_2,having a school uniform can reduce bullying as...,School uniforms encourage discipline or focus ...,We should abandon the use of school uniform,-1
3,arg_4_0,kp_4_3,having a school uniform can reduce bullying as...,School uniforms saves costs,We should abandon the use of school uniform,-1
4,arg_4_0,kp_4_4,having a school uniform can reduce bullying as...,School uniforms create a sense of equality/unity,We should abandon the use of school uniform,-1


In [ ]:
pred.to_csv('/content/drive/MyDrive/EMNLP_Argmining2021/dataset/predictions.csv')